# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lon,Max Temp in °C,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,port-aux-francais,-49.3500,70.2167,3.92,98,100,15.45,TF,1726083815
1,1,saint-joseph,-21.3667,55.6167,20.21,82,13,2.37,RE,1726083816
2,2,isafjordur,66.0755,-23.1240,3.76,68,5,1.27,IS,1726083817
3,3,najaf,31.9985,44.3398,38.88,15,0,2.57,IQ,1726083543
4,4,bayganin,48.6947,55.8742,19.61,36,84,5.46,KZ,1726083819


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [17]:
%%capture --no-display

# Configure the map plot
# YOUR CODE HERE

import random

def random_color():
    return "#{:06x}".format(random.randint(0, 0xFFFFFF))

unique_cities = city_data_df['City'].unique()
city_colors = {city: random_color() for city in unique_cities}
city_data_df['CityColor'] = city_data_df['City'].map(city_colors)

map_plot = city_data_df.hvplot.points(
    "Lon",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    size="Humidity",
    scale = 0.6,
    color='CityColor',
    alpha = 0.60
)


# Display the map
# YOUR CODE HERE
map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lon,Lat]   (CityColor,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [18]:
# Narrow down cities that fit criteria and drop any results with null values
# YOUR CODE HERE

perfect_temp_min = 20
perfect_temp_max = 38
perfect_humidity_max = 50
perfect_wind_speed_max = 2.00

# Drop any rows with null values
# YOUR CODE HERE

perfect_cities = city_data_df[
    (city_data_df["Max Temp in °C"] >= perfect_temp_min) &
    (city_data_df["Max Temp in °C"] <= perfect_temp_max) &
    (city_data_df["Humidity"] <= perfect_humidity_max) &
    (city_data_df["Wind Speed"] <= perfect_wind_speed_max)
].dropna()

# Display sample data
# YOUR CODE HERE

perfect_cities.head()

,City_ID,City,Lat,Lon,Max Temp in °C,Humidity,Cloudiness,Wind Speed,Country,Date,CityColor
65,65,albany,42.6001,-73.9662,25.45,45,59,0.89,US,1726083894,#0d8494
87,87,bethel,41.3712,-73.4140,25.36,48,0,0.45,US,1726083920,#e61dc3
130,130,sao francisco de assis,-29.5503,-55.1311,31.02,50,100,1.50,BR,1726083974,#f95734
153,153,tamanrasset,22.7850,5.5228,30.95,33,40,1.54,DZ,1726083849,#e447ad
237,237,turpan,42.9333,89.1667,25.47,18,92,1.25,CN,1726084107,#85d01e


### Step 3: Create a new DataFrame called `hotel_df`.

In [19]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
# YOUR CODE HERE

hotel_df = perfect_cities[["City", "Country", "Lat", "Lon", "Humidity"]].copy()


# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
# YOUR CODE HERE

hotel_df["Hotel Name"] = None

# Display sample data
# YOUR CODE HERE

hotel_df.head()

,City,Country,Lat,Lon,Humidity,Hotel Name
65,albany,US,42.6001,-73.9662,45,None
87,bethel,US,41.3712,-73.4140,48,None
130,sao francisco de assis,BR,-29.5503,-55.1311,50,None
153,tamanrasset,DZ,22.7850,5.5228,33,None
237,turpan,CN,42.9333,89.1667,18,None


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [20]:
# Set parameters to search for a hotel
#YOUR CODE HERE

import json
import requests

api_key = geoapify_key


categories= "accommodation.hotel"

radius = 10000

limit= 10

params = {
    "categories":categories,
    "limit":limit,
    "apiKey":api_key    
}
          

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    
    # get latitude, longitude from the DataFrame
    # YOUR CODE HERE
    
    latitude = row["Lat"]
    longitude = row["Lon"]
   
    
    # Add the current city's latitude and longitude to the params dictionary
    
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"

    # Set base URL
    
    url = "https://api.geoapify.com/v2/places"

    places_url = f"{url}"
    
   #Convert the API response to JSON format  

    response = requests.get(places_url, params = params)

    data = response.json()


    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = data["features"][0]["properties"]["name"]
    except(KeyError, IndexError):
        #If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
#Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df.head()

Starting hotel search
albany - nearest hotel: No hotel found
bethel - nearest hotel: Hampton Inn Danbury
sao francisco de assis - nearest hotel: Hotel Aguiar
tamanrasset - nearest hotel: فندق الأمان
turpan - nearest hotel: Oriental
saurimo - nearest hotel: Hotel Princesinha
spearfish - nearest hotel: Best Western Black Hills Lodge
kabalo - nearest hotel: No hotel found
jamestown - nearest hotel: DoubleTree Jamestown
barkhan - nearest hotel: No hotel found
goshen - nearest hotel: Best Western Inn
kiffa - nearest hotel: فندق كيفة


,City,Country,Lat,Lon,Humidity,Hotel Name
65,albany,US,42.6001,-73.9662,45,No hotel found
87,bethel,US,41.3712,-73.4140,48,Hampton Inn Danbury
130,sao francisco de assis,BR,-29.5503,-55.1311,50,Hotel Aguiar
153,tamanrasset,DZ,22.7850,5.5228,33,فندق الأمان
237,turpan,CN,42.9333,89.1667,18,Oriental


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [22]:
%%capture --no-display

# Configure the map plot
# YOUR CODE HERE

def random_color():
    return "#{:06x}".format(random.randint(0, 0xFFFFFF))

unique_cities = hotel_df['City'].unique()
city_colors = {city: random_color() for city in unique_cities}
hotel_df['CityColor'] = hotel_df['City'].map(city_colors)

map_plot2 = hotel_df.hvplot.points(
    "Lon",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    size="Humidity",
    scale = 0.6,
    color='CityColor',
    alpha = 0.60,
    hover_cols=['Hotel Name', 'Country']
)


# Display the map
# YOUR CODE HERE

map_plot2

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lon,Lat]   (CityColor,Humidity,Hotel Name,Country)